# Japan Life Expectancy

In [2]:
import pandas as pd

In [5]:
# path to the .csv file 
data = "Japan_life_expectancy.csv"

# read the .csv file and create a dataframe
df = pd.read_csv(data)

In [8]:
for column in df.columns:
    print(column)

Prefecture
Life_expectancy
Physician
Junior_col
University
Public_Hosp
Pshic_hosp
Beds_psic
Nurses
Avg_hours
Salary
Elementary_school
Sport_fac
Park
Forest
Income_per capita
Density_pop
Hospitals
Beds
Ambulances
Health_exp
Educ_exp
Welfare_exp
